In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_excel('cert.xlsx')

In [3]:
dataset.shape

(150, 5)

In [4]:
dataset=np.array(dataset)

In [5]:
description = []
severity = []
scores = []
impact = []


for i in range(dataset.shape[0]):
        severity.append(dataset[i][4])
        scores.append(dataset[i][3])
        description.append(dataset[i][2])
        impact.append(dataset[i][1])

In [6]:
from bs4 import BeautifulSoup  
import re
from nltk.corpus import stopwords
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [7]:
# Get the number of reviews based on the dataframe column size

# Initialize an empty list to hold the clean reviews
clean_description = []
clean_impact = []
clean_severity = []
# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(150):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_description.append( review_to_words( description[i] ) )
    clean_impact.append( review_to_words( impact[i] ) )
    clean_severity.append( review_to_words( severity[i] ) )

In [8]:
clean_severity=np.array(clean_severity)

In [9]:
clean_description_array = np.array(clean_description)
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
description_features = vectorizer.fit_transform(clean_description_array)

# Numpy arrays are easy to work with, so convert the result to an 
# array
description_features = description_features.toarray()

Creating the bag of words...



In [10]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
n_folds = 5
score = 0.0
skf = StratifiedKFold(severity, n_folds)
avg_score = 0

for train_index, test_index in skf:
    X_train, X_test = description_features[train_index], description_features[test_index]
    y_train, y_test = clean_severity[train_index], clean_severity[test_index]
    forest = RandomForestClassifier(n_estimators = 50)
    forest.fit( X_train, y_train )
    score = forest.score(X_test,y_test)
    avg_score += score 
    print(score)
    
print("avg",avg_score/n_folds)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.6129032258064516
0.5806451612903226
0.5483870967741935
0.5517241379310345
0.42857142857142855
avg 0.5444462100746861


In [11]:
clean_description = np.array(clean_description)
clean_impact = np.array(clean_impact)
imp_desc = np.array(clean_description)

In [12]:
for i in range(150):
    imp_desc[i] = clean_description[i]+clean_impact[i]

In [15]:
clean_description[4]

'unprotected transport credentials issue discovered abb ellipse ellipse released prior december including ellipse select vulnerability exists authentication ellipse ldap ad using ldap protocol attacker could exploit vulnerability sniffing local network traffic allowing discovery authentication credentials'

In [70]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
description_features = vectorizer.fit_transform(imp_desc)

# Numpy arrays are easy to work with, so convert the result to an 
# array
description_features = description_features.toarray()

Creating the bag of words...



In [71]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
n_folds = 5
score = 0.0
skf = StratifiedKFold(severity, n_folds)
avg_score = 0

for train_index, test_index in skf:
    X_train, X_test = description_features[train_index], description_features[test_index]
    y_train, y_test = clean_severity[train_index], clean_severity[test_index]
    forest = RandomForestClassifier(n_estimators = 50)
    forest.fit( X_train, y_train )
    score = forest.score(X_test,y_test)
    avg_score += score 
    print(score)
    
print("avg",avg_score/n_folds)

0.6129032258064516
0.6129032258064516
0.5161290322580645
0.41379310344827586
0.5714285714285714
avg 0.545431431749563
